<a href="https://colab.research.google.com/github/TheNeodev/RVC-MAKER/blob/main/webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RVC MAKER 🎵

**This project was created by [TheNeoDev](https://github.com/TheNeodev)** base on [PhamHuynhAnh16/Vietnamese-RVC](https://github.com/PhamHuynhAnh16/Vietnamese-RVC)

---



<big>**Please do not use the project for any purposes that violate ethics, laws, or cause harm to individuals, organizations...**

---
 **In case the user does not comply with the terms or violates them, I will not be responsible for any claims, damages, or legal liabilities, whether in contract, due to negligence, or other reasons, arising from, out of, or related to the software, the use of the software, or other transactions related to the software.**</big>

In [ ]:
#@title **🌟Settings**

import os
from ipywidgets import Button
from IPython.display import clear_output
print("👩🏻‍💻 INSTALLING...")

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

!git clone https://github.com/TheNeodev/RVC-MAKER.git /content/RVC-MAKER > /dev/null 2>&1
!pip install -r /content/RVC-MAKER/requirements.txt --no-cache-dir -q > /dev/null 2>&1
!pip install pyngrok --no-cache-dir -q > /dev/null 2>&1

#@markdown **💻 Installation will take about 2 minutes to complete!**
clear_output()
Button(description="\u2714 done!", button_style="success")

In [ ]:
#@title **Run webui**
import os
import shutil

%cd /content/RVC-MAKER
#@markdown **To experience all the features, use the interface :)**

#@markdown **If you know you can use the tensorboard to check for overtraining 👍**
tensorboard = False #@param {type:"boolean"}





# @markdown  ### Choose a sharing method:

import codecs
import threading
import urllib.request
import time
import ipywidgets as widgets
from IPython.display import display
import os
from pyngrok import ngrok
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

method = "gradio"  # @param ["gradio", "localtunnel", "ngrok"]


#@markdown If you selected the 'ngrok' method, obtain your auth token here: https://dashboard.ngrok.com/get-started/your-authtoken
ngrok_token = "" # @param {type:"string"}

def start_gradio():
    !python main/app/app.py --share

def start_localtunnel():
    !npm install -g localtunnel &>/dev/null
    with open('url.txt', 'w') as file:
        file.write('')
    get_ipython().system_raw('lt --port 7860 >> url.txt 2>&1 &')
    time.sleep(2)
    endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

    with open('url.txt', 'r') as file:
        tunnel_url = file.read()
        tunnel_url = tunnel_url.replace("your url is: ", "")

    print(f"Share Link: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m\n")

    password_endpoint_widget = widgets.Text(
    value=endpoint_ip,
    description='Password IP:',
    disabled=True
    )
    display(password_endpoint_widget)
    !python main/app/app.py


def start_ngrok():
    try:
        ngrok.set_auth_token(ngrok_token)
        ngrok.kill()
        tunnel = ngrok.connect(7860)
        print(f"Ngrok URL: \033[0m\033[93m{tunnel.public_url}\033[0m", end="\033[0m\n")
        !python app.py --listen
    except Exception as e:
        print(f"Error starting ngrok: {e}")

def start_app():
    if method == 'gradio':
        start_gradio()
    elif method == 'localtunnel':
        start_localtunnel()
    elif method == 'ngrok':
        start_ngrok()

if tensorboard:
  %load_ext tensorboard
  %tensorboard --logdir ./assets/logs --port=6870


thread_app = threading.Thread(target=start_app)
thread_app.start()


while True:
    time.sleep(5)

# Extra

In [ ]:

#@title upload model to HF model
repo_hf = "NeoPy/TTS-G" # @param {type:"string"}
pth = "/content/RVC-MAKER/assets/weights/TTS_100e_500s.pth" # @param {type:"string"}
index = "/content/RVC-MAKER/assets/logs/TTS/added_IVF59_Flat_nprobe_1_TTS_v2.index" # @param {type:"string"}

#@markdown get token on https://huggingface.co/settings/tokens

token = "hf_" # @param {type:"string"}


import huggingface_hub
import zipfile
import os
def upload_model(repo, pth, index, token):
    """
    Upload a model to the Hugging Face Hub

    Args:
        repo: str, the name of the repository (including user/org, e.g., "username/repo_name")
        pth: str, path to the model file
        index: str, the index of the model in the repository
        token: str, the API token

    Returns:
        str, message indicating the success of the operation
    """
    readme = f"""
    # {repo}
    This is a model uploaded by RVC Maker, using [NeoDev](https://github.com/TheNeodev)'s script.
    """
    repo_name = repo.split('/')[1]
    with zipfile.ZipFile(f'{repo_name}.zip', 'w') as zipf:
        zipf.write(pth, os.path.basename(pth))
        zipf.write(index, os.path.basename(index))
        zipf.writestr('README.md', readme)

    # Corrected: Pass the repo_id directly as the first argument
    huggingface_hub.HfApi().create_repo(repo_id=repo, token=token, exist_ok=True)
    # Change 'path' to 'path_or_fileobj' as per the error and common usage in newer versions
    huggingface_hub.HfApi().upload_file(token=token, path_or_fileobj=f'{repo.split("/")[1]}.zip', repo_id=repo, path_in_repo=f'{repo.split("/")[1]}.zip')
    os.remove(f'{repo.split("/")[1]}.zip')
    return "Model uploaded successfully"


upload_model(repo_hf, pth, index, token)